In [1]:
import os
import sys
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

import pandas as pd
import numpy as np
import datetime
import torch
import math
import json

from Clust.clust.ML.regression_JS.ada_rnn import ada_rnn
from Clust.clust.preprocessing.imputation.basicMethod import BasicImputation
from Clust.clust.transformation.purpose.machineLearning import LSTMData

from Clust.clust.ingestion.influx import influx_client_v2 as influx_Client
from Clust.clust.data import df_set_data
from Clust.clust.data import data_interface

from Clust.setting import influx_setting_KETI as ins
db_client = influx_Client.InfluxClient(ins.CLUSTDataServer2)

from clust.preprocessing.dataPreprocessing import DataPreprocessing
from Clust.clust.preprocessing.imputation.basicMethod import BasicImputation
from Clust.clust.transformation.purpose.machineLearning import LSTMData

from Clust.clust.ML.tool import data as ml_data

# Set Param

In [2]:
feature_list = ["out_blacksphere_temp", "out_coai", "out_h2s", 'out_humi', 'out_nh3', 'out_pm10', 'out_temp', 'out_ultraviolet_rays']
target_col = "out_pm10"

get_y_data_transformParameter ={
    "future_step": 1,# 0~x
    "past_step":24,# 1~y
    "feature_col": feature_list,
    "target_col":target_col 
}   

param = {
    "x_features" : 8, 
    "class_num" : 1, 
    "dropout" : 0.0, 
    "len_seq" : 24, 
    "loss_type" : "adv",
    "hidden_size" : 64,
    "num_layers" : 2
}

args = {
    "output_folder_name" : "./ada_rnn_model",
    "save_model_name" : "test_adarnn_model_by_kweather_data",
    "lr" : 5e-4,
    "n_epochs" : 200,
    "num_layers" : 2, 
    "len_seq" : 24,
    "pre_epoch" : 40,
    "len_win" : 0,
    "dw" : 0.5,
    "early_stop" :40
}
k = 2

train_start_time = "2021-01-01 00:00:00"
train_end_time = "2021-12-31 00:00:00"
bk_name = "air_outdoor_kweather"
ms_name = "OC3CL200010"

#test_start_time = "2022-01-01 00:00:00"
#test_end_time = "2022-04-29 00:00:00"

# get Data

## Data Read

In [3]:
origin_data = db_client.get_data_by_time(train_start_time, train_end_time, bk_name, ms_name)

In [4]:
origin_data

,out_blacksphere_temp,out_co,out_coai,out_coci,out_h2s,out_humi,out_light_intensity,out_nh3,out_no2,out_noise,out_o3,out_pm10,out_pm25,out_so2,out_temp,out_ultraviolet_rays
time,,,,,,,,,,,,,,,,
2021-01-01 00:00:00+00:00,-8.9,-9999.0,96.0,96.0,0.20,66.0,-9999.0,0.0,-9999.0,50.0,-9999.0,10.0,7.0,-9999.0,-9.31,0.0
2021-01-01 00:02:00+00:00,-8.9,-9999.0,96.0,96.0,0.21,66.0,-9999.0,0.0,-9999.0,50.0,-9999.0,11.0,7.0,-9999.0,-9.31,0.0
2021-01-01 00:04:00+00:00,-8.9,-9999.0,97.0,97.0,0.24,66.0,-9999.0,0.0,-9999.0,50.0,-9999.0,10.0,6.0,-9999.0,-9.20,0.0
2021-01-01 00:06:00+00:00,-8.9,-9999.0,96.0,96.0,0.26,66.0,-9999.0,0.0,-9999.0,50.0,-9999.0,10.0,7.0,-9999.0,-9.20,0.0
2021-01-01 00:08:00+00:00,-8.9,-9999.0,96.0,96.0,0.25,66.0,-9999.0,0.0,-9999.0,50.0,-9999.0,10.0,7.0,-9999.0,-9.20,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-20 09:06:00+00:00,23.0,-9999.0,97.0,97.0,0.00,79.0,-9999.0,0.0,-9999.0,43.0,-9999.0,9.0,6.0,-9999.0,25.30,1.3
2021-08-20 09:08:00+00:00,23.1,-9999.0,97.0,97.0,0.00,80.0,-9999.0,0.0,-9999.0,43.0,-9999.0,9.0,6.0,-9999.0,25.30,1.3
2021-08-20 09:10:00+00:00,23.1,-9999.0,97.0,97.0,0.00,79.0,-9999.0,0.0,-9999.0,48.0,-9999.0,9.0,6.0,-9999.0,25.40,1.3


In [5]:
origin_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 157951 entries, 2021-01-01 00:00:00+00:00 to 2021-08-20 09:14:00+00:00
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   out_blacksphere_temp  157951 non-null  float64
 1   out_co                157951 non-null  float64
 2   out_coai              157951 non-null  float64
 3   out_coci              157951 non-null  float64
 4   out_h2s               157951 non-null  float64
 5   out_humi              157951 non-null  float64
 6   out_light_intensity   157951 non-null  float64
 7   out_nh3               157951 non-null  float64
 8   out_no2               157951 non-null  float64
 9   out_noise             157951 non-null  float64
 10  out_o3                157951 non-null  float64
 11  out_pm10              157951 non-null  float64
 12  out_pm25              157951 non-null  float64
 13  out_so2               157951 non-null  float64
 14  out_te

In [6]:
# Check Frequency
freq = db_client.get_freq(bk_name, ms_name)
print(freq)

0 days 00:02:00


In [7]:
origin_data = origin_data[feature_list]

## Data Refine

In [8]:
# fill in the missing time index
refine_param = {'removeDuplication': {'flag': True} , 'staticFrequency': {'flag': True, 'frequency': "1H"}}
data_refine = DataPreprocessing().get_refinedData(origin_data, refine_param)

In [9]:
data_refine.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5554 entries, 2021-01-01 00:00:00+00:00 to 2021-08-20 09:00:00+00:00
Freq: H
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   out_blacksphere_temp  5547 non-null   float64
 1   out_coai              5547 non-null   float64
 2   out_h2s               5547 non-null   float64
 3   out_humi              5547 non-null   float64
 4   out_nh3               5547 non-null   float64
 5   out_pm10              5547 non-null   float64
 6   out_temp              5547 non-null   float64
 7   out_ultraviolet_rays  5547 non-null   float64
dtypes: float64(8)
memory usage: 390.5 KB


In [10]:
data_refine.head()

,out_blacksphere_temp,out_coai,out_h2s,out_humi,out_nh3,out_pm10,out_temp,out_ultraviolet_rays
time,,,,,,,,
2021-01-01 00:00:00+00:00,-8.976667,96.100000,0.289333,65.933333,0.0,10.433333,-9.322000,0.000000
2021-01-01 01:00:00+00:00,-9.249259,97.111111,0.290741,66.925926,0.0,8.407407,-9.788889,0.003704
2021-01-01 02:00:00+00:00,-9.418966,97.896552,0.306897,67.620690,0.0,6.965517,-10.069310,0.003448
2021-01-01 03:00:00+00:00,-9.534483,98.000000,0.295517,68.344828,0.0,6.172414,-10.310345,0.000000
2021-01-01 04:00:00+00:00,-9.471786,98.000000,0.272857,67.785714,0.0,6.000000,-10.178929,0.060714


## Data Nan -> Mean

In [11]:
method = "mean"
max_limit = {}
parameter = {}

data_refine_nan = BasicImputation(data_refine, method, max_limit, parameter).simpleMethod()

In [12]:
data_refine_nan.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5554 entries, 2021-01-01 00:00:00+00:00 to 2021-08-20 09:00:00+00:00
Freq: H
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   out_blacksphere_temp  5554 non-null   float64
 1   out_coai              5554 non-null   float64
 2   out_h2s               5554 non-null   float64
 3   out_humi              5554 non-null   float64
 4   out_nh3               5554 non-null   float64
 5   out_pm10              5554 non-null   float64
 6   out_temp              5554 non-null   float64
 7   out_ultraviolet_rays  5554 non-null   float64
dtypes: float64(8)
memory usage: 390.5 KB


## Data Scale

In [13]:
# set parameter
scaler_root_dir ='./scaler/'

clean_train_data_param = 'NoClean'

data_name_x = "AdaRNN_data_x"
feature_list_x= list(data_refine_nan.columns)

split_ratio = 0.8
scaler_param='scale'
scale_method='minmax'

In [14]:
scaler_root_path_x = os.path.join(scaler_root_dir, data_name_x, clean_train_data_param)
train_x, val_x, X_scaler_file_path = ml_data.get_train_val_data(data_refine_nan, feature_list_x, scaler_root_path_x, split_ratio, 
                                                                scaler_param, scale_method, mode = 'Classification')

New json file is created from data.json file
['out_blacksphere_temp', 'out_coai', 'out_h2s', 'out_humi', 'out_nh3', 'out_pm10', 'out_temp', 'out_ultraviolet_rays']
Make New scaler File


In [15]:
X_scaler_file_path

'./scaler/AdaRNN_data_x/NoClean/minmax/4457f465de3d0e17dd9f547db5b454b7/scaler.pkl'

In [16]:
import json
scale_path_dict = {"scale_path" : X_scaler_file_path}
with open('./scale_path_dict.json','w') as f:
    json.dump(scale_path_dict, f, ensure_ascii=False, indent=4)

## get Data y

In [17]:
LSTMD = LSTMData()
train_x_arr, train_y_arr = LSTMD.transform_Xy_arr(train_x, get_y_data_transformParameter, "NoClean")
valid_x_arr, valid_y_arr = LSTMD.transform_Xy_arr(val_x, get_y_data_transformParameter, "NoClean")

1. Original Data Lenagh: 4463
2. Removed Data Length: 0
3. Clean Data Leangth: 4440
1. Original Data Lenagh: 1089
2. Removed Data Length: 0
3. Clean Data Leangth: 1066


In [18]:
train_x_arr.shape, train_y_arr.shape, valid_x_arr.shape, valid_y_arr.shape

((4440, 24, 8), (4440, 1), (1066, 24, 8), (1066, 1))

In [19]:
train_y_arr = train_y_arr.reshape(train_y_arr.shape[0],)
valid_y_arr = valid_y_arr.reshape(valid_y_arr.shape[0])

# Data Loader

In [20]:
adarnn = ada_rnn.ClustAdaRnn(param)

{'x_features': 8, 'class_num': 1, 'dropout': 0.0, 'len_seq': 24, 'loss_type': 'adv', 'hidden_size': 64, 'num_layers': 2}


In [21]:
train_loader_list, valid_loader = adarnn.create_trainloader(train_x_arr, train_y_arr, valid_x_arr, valid_y_arr, k, 
                                                            train_start_time, train_end_time)

# Training

In [22]:
## training
adarnn.train(train_loader_list, valid_loader, args)

[2023-03-21 15:32:30] - create model...
[2023-03-21 15:32:30] - Epoch: 0
[2023-03-21 15:32:30] - training...


100%|██████████████████████████████████████████| 3/3 [00:00<00:00,  4.41it/s]

[2023-03-21 15:32:31] - evaluating...



Valid: 100%|████████████████████████████████| 17/17 [00:00<00:00, 581.89it/s]

[2023-03-21 15:32:31] - train 0.125825, valid MSE Loss 0.006824, valid L1 Loss 0.081323
[2023-03-21 15:32:31] - Epoch: 1
[2023-03-21 15:32:31] - training...


...

[2023-03-21 15:32:50] - train 0.013136, valid MSE Loss 0.076973, valid L1 Loss 0.277232
[2023-03-21 15:32:50] - Epoch: 80
[2023-03-21 15:32:50] - training...



100%|██████████████████████████████████████████| 3/3 [00:00<00:00, 17.14it/s]

[2023-03-21 15:32:50] - evaluating...



Valid: 100%|████████████████████████████████| 17/17 [00:00<00:00, 640.18it/s]

[2023-03-21 15:32:50] - train 0.014266, valid MSE Loss 0.095658, valid L1 Loss 0.309098
[2023-03-21 15:32:50] - early stop
[2023-03-21 15:32:50] - best val mse loss score: 0.00017861976555329473 @ 40
[2023-03-21 15:32:50] - Finished.
